In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# load the datasets
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
# divide price / 1000 
train["price"] = train["price"] / 1000
test["price"] = test["price"] / 1000

In [4]:
# drop unwanted columns 
train = train.drop(columns=["Unnamed: 0", "zipcode"])
test = test.drop(columns=["Unnamed: 0", "zipcode", "id", "date"])

In [5]:
print(train.columns)

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')


In [6]:
print(test.columns)

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')


In [7]:
# separate features (X) and target variable (y)
y_train = train["price"]
X_train = train.drop(columns=["price"])

In [8]:
# separate features (X) and target variable (y)
y_test = test["price"]
X_test = test.drop(columns=["price"])

In [9]:
# save featrue names before scaling (for coefficients later)
feature_names = X_train.columns

In [10]:
# scale the data so that each feature has mean 0 and standard deviation 1
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Problem 2

### Part 1 & 2

In [11]:
model = LinearRegression()
model.fit(X_train, y_train)

# generate predictions for training and testing data
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# compute MSE and R^2 for training data
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# compute MSE and R^2 for testing data
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Train MSE:", train_mse)
print("Train R2:", train_r2)
print("Test MSE:", test_mse)
print("Test R2:", test_r2)

Train MSE: 31486.16777579488
Train R2: 0.7265334318706018
Test MSE: 57628.15470567038
Test R2: 0.6543560876120955


In [12]:
# obtain coefficients for the model
coefficients = pd.DataFrame({
    "Feature": feature_names,
    "Coefficient": model.coef_
})

coefficients_sorted = coefficients.reset_index(drop=True)
print(coefficients_sorted)

          Feature  Coefficient
0        bedrooms   -12.521962
1       bathrooms    18.527633
2     sqft_living    56.748837
3        sqft_lot    10.881868
4          floors     8.043721
5      waterfront    63.742900
6            view    48.200109
7       condition    12.964269
8           grade    92.231475
9      sqft_above    48.290089
10  sqft_basement    27.137032
11       yr_built   -67.643117
12   yr_renovated    17.271380
13            lat    78.375737
14           long    -1.035203
15  sqft_living15    45.577658
16     sqft_lot15   -12.930091


## Problem 3

In [13]:
# add column of ones to include intercept term
X_train_bias = np.c_[np.ones(X_train.shape[0]), X_train]
X_test_bias = np.c_[np.ones(X_test.shape[0]), X_test]

In [14]:
# compute closed-form solution
theta = np.linalg.inv(X_train_bias.T @ X_train_bias) @ X_train_bias.T @ y_train

In [15]:
# function to predict the response on a new testing point
def predict(X, theta):
    return X @ theta

In [16]:
# generate predictions on training and testing data using closed-form solution
y_train_pred_cf = predict(X_train_bias, theta)
y_test_pred_cf = predict(X_test_bias, theta)

In [17]:
# compute MSE and R2 for closed-form solution on training data
train_mse_cf = mean_squared_error(y_train, y_train_pred_cf)
train_r2_cf = r2_score(y_train, y_train_pred_cf)

In [18]:
# compute MSE and R2 for closed-form solution on testing data
test_mse_cf = mean_squared_error(y_test, y_test_pred_cf)
test_r2_cf = r2_score(y_test, y_test_pred_cf)

In [19]:
print(train_mse_cf, train_r2_cf)
print(test_mse_cf, test_r2_cf)

34144.195361988975 0.7034476855465108
56177.34019266813 0.6630578273603909


## Problem 4

### Part 1

In [20]:
# construct polynomial feature matrix
def build_poly_features(X, p):
    X_poly = X
    for degree in range(2, p + 1):
        X_poly = np.c_[X_poly, X**degree]
    return X_poly


In [21]:
# train polynomial regression using close-form 
def train_polynomial_regression(X, y, p):
    # build polynomial features
    X_poly = build_poly_features(X, p)
    
    # add bias term
    X_poly_bias = np.c_[np.ones(X_poly.shape[0]), X_poly]
    
    # closed-form solution
    theta = np.linalg.inv(X_poly_bias.T @ X_poly_bias) @ X_poly_bias.T @ y
    
    return theta


In [22]:
# generate predictions using trained polynomical model
def predict_polynomial(X, theta, p):
    X_poly = build_poly_features(X, p)
    X_poly_bias = np.c_[np.ones(X_poly.shape[0]), X_poly]
    return X_poly_bias @ theta


### Part 2

In [23]:
# get feature names
feature_names = train.drop(columns=["price"]).columns

#find index of 'sqft_lving'
sqft_index = list(feature_names).index("sqft_living")

In [24]:
# extract scaled sqft-living column 
X_train_sqft = X_train[:, sqft_index].reshape(-1, 1)
X_test_sqft = X_test[:, sqft_index].reshape(-1, 1)

In [25]:
degrees = [1, 2, 5]

results = []

for p in degrees:
    
    # train model of degree p
    theta_poly = train_polynomial_regression(X_train_sqft, y_train, p)
    
    # generate predictions on training and testing data
    y_train_pred = predict_polynomial(X_train_sqft, theta_poly, p)
    y_test_pred = predict_polynomial(X_test_sqft, theta_poly, p)
    
    # compute MSE/R2
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    results.append([p, train_mse, train_r2, test_mse, test_r2])


In [26]:
results_df = pd.DataFrame(
    results,
    columns=["Degree", "Train MSE", "Train R2", "Test MSE", "Test R2"]
)

print(results_df)

   Degree     Train MSE  Train R2       Test MSE   Test R2
0       1  57947.526161  0.496709   88575.978543  0.468736
1       2  54822.665116  0.523849   71791.679479  0.569406
2       5  52626.111955  0.542927  570616.914821 -2.422464


## Question 5

### Part 1

In [27]:
# add column of ones to include intercept term in gradient decent 
X_train_bias = np.c_[np.ones(X_train.shape[0]), X_train]
X_test_bias = np.c_[np.ones(X_test.shape[0]), X_test]

In [28]:
# train linear regression model using gradient descent
def gradient_descent(X, y, alpha, iterations):
    N, d = X.shape
    theta = np.zeros(d)
    
    for i in range(iterations):
        gradient = (2/N) * X.T @ (X @ theta - y)
        theta = theta - alpha * gradient
    
    return theta


### Part 2

In [29]:
learning_rates = [0.01, 0.1, 0.5]
iterations_list = [10, 50, 100]

results_gd = []

for alpha in learning_rates:
    for iters in iterations_list:

        # train model using gradinet descent with given alpha and iterations
        theta_gd = gradient_descent(X_train_bias, y_train, alpha, iters)

        # generate predictions on training and testing data
        y_train_pred = X_train_bias @ theta_gd
        y_test_pred = X_test_bias @ theta_gd

        # compute mse/R2
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_r2 = r2_score(y_train, y_train_pred)
        
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        
        results_gd.append([alpha, iters, train_mse, train_r2, test_mse, test_r2])


In [30]:
results_gd_df = pd.DataFrame(
    results_gd,
    columns=["Alpha", "Iterations", "Train MSE", "Train R²", "Test MSE", "Test R²"]
)

results_gd_df = results_gd_df.round(3)

pd.set_option("display.width", 1000)
pd.set_option("display.max_columns", None)

display(results_gd_df)

,Alpha,Iterations,Train MSE,Train R²,Test MSE,Test R²
0,0.01,10,2.357278e+05,-1.047000e+00,2.805687e+05,-6.830000e-01
1,0.01,50,6.972050e+04,3.940000e-01,9.704954e+04,4.180000e-01
2,0.01,100,3.682035e+04,6.800000e-01,6.333304e+04,6.200000e-01
3,0.10,10,3.510510e+04,6.950000e-01,6.163043e+04,6.300000e-01
4,0.10,50,3.149726e+04,7.260000e-01,5.772247e+04,6.540000e-01
5,0.10,100,3.148643e+04,7.270000e-01,5.763896e+04,6.540000e-01
6,0.50,10,1.456064e+17,-1.264635e+12,1.626068e+17,-9.752880e+11
7,0.50,50,1.259542e+67,-1.093949e+62,1.406601e+67,-8.436553e+61
8,0.50,100,3.322792e+129,-2.885942e+124,3.710745e+129,-2.225642e+124


## Problem 6

### Part 2

In [31]:
# train ridge regression modle using gradient descent 
def ridge_gradient_descent(X, y, alpha, iterations, lam):
    N, d = X.shape
    theta = np.zeros(d)

    for _ in range(iterations):
        grad_mse = (2/N) * (X.T @ (X @ theta - y))
        
        grad_ridge = 2 * lam * theta
        grad_ridge[0] = 0

        theta -= alpha * (grad_mse + grad_ridge)

    return theta


### Part 3

In [32]:
np.random.seed(0)
N = 1000

# simulate feature values uniformly from [-2, 2]
X = np.random.uniform(-2, 2, size=N).reshape(-1, 1)

# generate Gaussian noise with variance 2
e = np.random.normal(0, np.sqrt(2), size=N)

# generate response: y = 1 + 2x + noise
y = 1 + 2*X.flatten() + e

# add bias column for intercept
Xb = np.c_[np.ones(N), X]

# closed-form ridge regression solution
def ridge_closed_form(Xb, y, lam):
    d = Xb.shape[1]
    
    # identity matrix 
    I = np.eye(d)
    I[0, 0] = 0  
    
    # compute ridge solution
    theta = np.linalg.inv(Xb.T @ Xb + lam * I) @ (Xb.T @ y)
    return theta

ridge_results = []

# Ordinary Least Squares 
theta_ols = ridge_closed_form(Xb, y, lam=0)
yhat_ols = Xb @ theta_ols

ridge_results.append(["OLS (0)", theta_ols[1],
                      mean_squared_error(y, yhat_ols),
                      r2_score(y, yhat_ols)])

# ridge regression for different λ values
lambdas = [1, 10, 100, 1000, 10000]

for lam in lambdas:
    theta = ridge_closed_form(Xb, y, lam)
    yhat = Xb @ theta
    
    ridge_results.append([lam, theta[1],
                          mean_squared_error(y, yhat),
                          r2_score(y, yhat)])


In [33]:
ridge_df = pd.DataFrame(
    ridge_results,
    columns=["Lambda", "Slope", "MSE", "R²"]
)

ridge_df = ridge_df.round(4)

display(ridge_df)



,Lambda,Slope,MSE,R²
0,OLS (0),1.9657,1.8654,0.7368
1,1,1.9642,1.8654,0.7368
2,10,1.9513,1.8657,0.7367
3,100,1.8302,1.8902,0.7333
4,1000,1.1296,2.8098,0.6035
5,10000,0.2340,5.9173,0.1650
